In [2]:
from functools import partial

from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载arXiv上的论文ReAct: Synergizing Reasoning and Acting in Language Models
loader = ArxivLoader(query = "2210.03629", load_max_docs = 1)
docs = loader.load()

# 把文本分割成500个字符为一组的片段
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
chunks = text_splitter.split_documents(docs)

llm = ChatOllama(model = 'qwen2.5')

# 构建工具函数：将Document转换成字符串
document_prompt = PromptTemplate.from_template('{page_content}')
partial_format_document = partial(format_document, prompt = document_prompt)

# 构建Map链，对每个文档都先进行一轮总结
map_chain = (
    {'context': partial_format_document}
    | PromptTemplate.from_template('Summarize this content(用中文): \n\n {context}')
    | llm
    | StrOutputParser()
)

# 构建Reduce链，合并之前的所有总结内容
reduce_chain = (
    {'context': lambda strs: '\n\n'.join(strs)}
    | PromptTemplate.from_template('Combine these summaries(用中文): \n\n{context}')
    | llm
    | StrOutputParser()
)

# 把两个链合并成MapReduce链
map_reduce = map_chain.map() | reduce_chain
map_reduce.invoke(chunks[:4], config = {'max_concurrency': 5})

'该内容摘要如下：\n\n一篇发表于ICLR 2023会议的论文介绍了REACT模型，旨在提升语言模型在推理与行动方面的表现。作者包括Shunyu Yao、Jeffrey Zhao、Dian Yu、Nan Du、Izhak Shafran、Karthik Narasimhan和Yuan Cao，分别来自普林斯顿大学计算机科学系和Google研究团队。该方法名为ReAct，结合了异常处理和动作执行的功能，通过与外部来源（如知识库或环境）交互来识别错误并获取更多信息。研究者将ReAct应用于多种语言理解和决策任务，并展示了其在基准模型上的优越性能，特别是在可解释性和可信度方面。\n\n此外，该内容还指出，ReAct能够生成比没有推理痕迹的基础模型更具可解释性的类人类任务解决轨迹。在两个互动决策基准（ALFWorld和WebShop）上，ReAct分别以绝对成功率34%和10%优于模仿学习和强化学习方法，即使是在没有详细提示的情况下。\n\n总结如下：\n\nREACT模型通过与外部资源交互来改进大型语言模型的推理与行动能力。该研究展示了ReAct在多个任务中的优越表现，并提高了模型的可解释性和可信度。特别是在互动决策任务中，ReAct表现出显著的优势，超过了模仿学习和强化学习方法。'